In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import strategy.trendlabeling as tlb
import numpy as np
import datetime

import getdata as gd
import afml.filters as flt 
from afml.util.volatility import getDailyVolatility
import features.bars as bars

1. Import Dataset

In [2]:
df = gd.get_yf_data(tickers= "SPY AAPL ALGM DNOW", period='1y', interval='1d')
df = df[df['Ticker'] == 'ALGM']

[*********************100%***********************]  4 of 4 completed


In [3]:

df.index = pd.to_datetime(df.index, format="%Y-%m-%dT%H:%M:%S.%fZ") 
df = df[['Open', 'High', 'Low', 'Adj Close', 'Volume']]

In [4]:
# to do
# dollar bar threshold = 
# can try 30 day moving average of volume and then dividing by 50 
# meaning sample about 50 times a day 

#create bars    
dollar_bars = bars.get_dollar_bars(df, 5000000) #5,000,000 is an arbitrarily selected threshold for now

#create dataframe
df = pd.DataFrame(dollar_bars)

#view first five entries
df.head()

KeyError: 0

2. Form Dollar Bars

In [ ]:
#to do

3. Run cusum filter to get std dev of daily returns at equal threshold

In [ ]:
#to do

4. Add vertical barrier

In [ ]:
#to do

5. Apply Triple Barrier method where ptSl = [2,1]  (2 to 1 profit ratio)

In [ ]:
#to do

6. Generate label using getBins

In [ ]:
#to do

7. Drop underpopulated labels (snippet 3.8)

In [ ]:
#to do 

8. Adjust getBins to 0 if vertical barrier is touched first

In [ ]:
#to do

9. Apply strategy here to suggest the side, not the bet
    
    9.1 Trendlabeling

    9.2 Mean Reverting BB 

In [ ]:
#to do

10. Train model to decide trade or not trade since step 9 already decided a side (up or down)

    10.1 Random Forest

    10.2 SVC

In [ ]:
#to do

11. Compare model with or without Secondary Model

In [ ]:
# to do